In [18]:
import pandas as pd
import os
import pymongo
from datetime import datetime,date,time
import re
import json

In [2]:
df = pd.read_excel('../data/self/Best Books Ever.xlsx')

In [3]:
def get_mongo_db():
    mongo_connection = os.environ["MONGODB_CONNECTION"]
    client = pymongo.MongoClient(mongo_connection)
    return client['recommendation']

In [4]:
db = get_mongo_db()
books_collection = db.get_collection('books')

In [5]:
data = df.copy(True)

In [6]:
data = data.rename(columns={"Book ID": "_id"})
data=data.drop(columns=["bookFormat", "edition", "firstPublishDate"])

In [7]:
def categorize_value(value):
    global unknown_formats
    
    if pd.isna(value):
        return 'NaN'
    if isinstance(value, datetime):
        return 'datetime'
    if isinstance(value, date):
      return 'date'
    if isinstance(value, int):
      return 'int'
    if isinstance(value, str):
        value = value.strip()
        # Regular expressions for different date formats
        patterns = {
            "month day year": r"^(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}(st|nd|rd|th)? \d{4}$",
        }
        for fmt, pattern in patterns.items():
            if re.match(pattern, value):
                return fmt
        return 'Unknown format'
    return f'Invalid type: {type(value).__name__}'

In [8]:
# Apply the function to the 'publishDate' column and categorize the values
data['value_category'] = data['publishDate'].apply(categorize_value)

# Count the occurrences of each category
category_counts = data['value_category'].value_counts()

print(category_counts)

value_category
month day year    43314
int                4362
datetime           3017
Unknown format      905
NaN                 880
Name: count, dtype: int64


In [9]:
# First, let's see how many rows we have initially
print(f"Initial number of rows: {len(data)}")

# Drop rows where publishDate is NaN
data = data.dropna(subset=['publishDate'])
print(f"Number of rows after dropping NaN values: {len(data)}")

# Drop rows where value_category is 'Unknown format'
data = data[data['value_category'] != 'Unknown format' ]
print(f"Number of rows after dropping unknown formats: {len(data)}")

# Print the value counts of the remaining categories
print("\nRemaining value categories:")
print(data['value_category'].value_counts())

# First, let's see how many rows we have initially
print(f"Final number of rows: {len(data)}")

Initial number of rows: 52478
Number of rows after dropping NaN values: 51598
Number of rows after dropping unknown formats: 50693

Remaining value categories:
value_category
month day year    43314
int                4362
datetime           3017
Name: count, dtype: int64
Final number of rows: 50693


In [10]:
def parse_date(value):
    if pd.isna(value):
        return pd.NaT
    
    if isinstance(value, datetime):
        return value.date()
    
    if isinstance(value, (int, float)):
        # Assuming the int/float is a Unix timestamp
        try:
            return pd.to_datetime(value, unit='s').date()
        except:
            return pd.NaT
    
    if isinstance(value, str):
        value = value.strip()
        # Try parsing with dateutil parser
        try:
            return pd.to_datetime(value).date()
        except:
            # If dateutil fails, try specific formats
            formats = [
                "%B %d %Y",  # e.g., "January 1 2023"
                "%B %d, %Y",  # e.g., "January 1, 2023"
                "%b %d %Y",   # e.g., "Jan 1 2023"
                "%Y-%m-%d",   # e.g., "2023-01-01"
                "%Y-%m-%d %H:%M:%S"  # e.g., "2023-01-01 12:00:00"
            ]
            for fmt in formats:
                try:
                    return datetime.strptime(value, fmt).date()
                except:
                    continue
    
    return pd.NaT

# Apply the function to the 'publishDate' column
data['publishDate_parsed'] = data['publishDate'].apply(parse_date)

# Drop rows where parsing failed (resulted in NaT)
data = data.dropna(subset=['publishDate_parsed'])

print(f"Number of successfully parsed dates: {len(data)}")

# Display a sample of the parsed dates
print("\nSample of parsed dates:")
print(data['publishDate_parsed'].head())

# If you want to reset the index of your DataFrame after dropping rows:
data = data.reset_index(drop=True)

Number of successfully parsed dates: 50693

Sample of parsed dates:
0    2008-09-14
1    2004-09-28
2    2006-05-23
3    2000-10-10
4    2006-09-06
Name: publishDate_parsed, dtype: object


In [11]:
count = data['publishDate_parsed'].apply(categorize_value).value_counts()
print(count)
data['publishDate_parsed'].sample(5)

publishDate_parsed
date    50693
Name: count, dtype: int64


26129    2014-10-14
37062    2010-11-16
22831    2016-06-07
29214    2012-05-30
17846    2008-04-08
Name: publishDate_parsed, dtype: object

In [12]:
data['publishDate_parsed'] = data['publishDate_parsed'].apply(lambda x: str(x.isoformat()))

In [13]:
data = data.drop(columns=['publishDate', 'value_category'])
data = data.rename(columns={'publishDate_parsed': 'publishDate'})

In [14]:
data['publishDate'].sample(10)

21397    1970-01-01
34948    2014-07-04
31207    2011-05-10
47110    1987-01-15
10973    1992-06-02
8793     2016-01-05
50627    2012-10-07
10572    2015-06-04
49108    2007-01-23
33892    1999-06-01
Name: publishDate, dtype: object

In [15]:
# Filter out rows where 'title' is a datetime.time object
data = data[~data['title'].apply(lambda x: isinstance(x, time))]

In [55]:
def parse_genre(genre):
  if(isinstance(genre, list)):
    return genre
  print(genre)
  genre = genre.replace("'", "\"")
  try:
    g = json.loads(genre)
    return g
  except Exception as e:
    print(e, "\n", genre, type(genre))
    raise e
  
data['genres'] = data['genres'].apply(parse_genre)

In [23]:
data['publisher'] = data['publisher'].replace({'NaN': ''})
data['publisher'].isnull().sum()

2540

In [24]:
data = data.drop(columns=['setting', 'series', 'awards'])

In [54]:
data['genres'].sample(10)

29903    [Mystery, Vampires, Urban Fantasy, Paranormal,...
31597           [Travel, Fiction, Young Adult, Literature]
49870    [Economics, Nonfiction, Politics, History, Bus...
20871    [Classics, Poetry, Russia, Fiction, Russian Li...
2201     [Fiction, Japan, Mystery, Thriller, Japanese L...
22130    [Poetry, Romance, Audiobook, LGBT, Nonfiction,...
1990     [Fantasy, Young Adult, Urban Fantasy, Paranorm...
20602                                                   []
39605    [Fantasy, M M Romance, Romance, Paranormal, Ma...
41437    [Fantasy, Anthologies, Short Stories, Fiction,...
Name: genres, dtype: object

In [56]:
# Prepare the bulk operations
bulk_operations = []
for d in data.to_dict('records'):
    bulk_operations.append(
        pymongo.UpdateOne(
            {"_id": d["_id"]},
            {"$set": d},
            upsert=True
        )
    )

# Execute the bulk operation
try:
    result = books_collection.bulk_write(bulk_operations)
    print(f"Modified {result.modified_count} documents")
    print(f"Upserted {result.upserted_count} documents")
except pymongo.BulkWriteError as bwe:
    print(f"Error in bulk write operation: {bwe.details}")
    # Optionally, you can still get stats on successful operations:
    if bwe.details.get('nMatched') is not None:
        print(f"Matched {bwe.details['nMatched']} documents")
    if bwe.details.get('nModified') is not None:
        print(f"Modified {bwe.details['nModified']} documents")
    if bwe.details.get('nUpserted') is not None:
        print(f"Upserted {bwe.details['nUpserted']} documents")

Modified 0 documents
Upserted 0 documents
